In [1]:
!kaggle competitions download -c ss3-hackathon-online-data-science-and-big-data


  0%|          | 0.00/2.10M [00:00<?, ?B/s]
 48%|████▊     | 1.00M/2.10M [00:01<00:01, 773kB/s]
 95%|█████████▌| 2.00M/2.10M [00:01<00:00, 1.51MB/s]
100%|██████████| 2.10M/2.10M [00:01<00:00, 1.37MB/s]


## import Library

In [2]:
!zip ss3-hackathon-online-data-science-and-big-data.zip

'zip' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
import numpy as np
import pandas as pd
import os 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import GammaRegressor
from sklearn.linear_model import PoissonRegressor
from sklearn.ensemble import RandomForestRegressor

from autogluon.tabular import TabularDataset, TabularPredictor
import autogluon.core as ag



In [3]:
# import csv with pandas 
df_train = pd.read_csv(r"Document\train_data.csv")
df_test = pd.read_csv(r"Document\test_data.csv")
df_train_target = pd.read_csv(r"Document\train_target.csv")

C:\Users\ASUS\AppData\Local\Temp/ipykernel_17684/4076387190.py:2: DtypeWarning: Columns (12,28,29,30,116,128) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv(r"Document\train_data.csv")
C:\Users\ASUS\AppData\Local\Temp/ipykernel_17684/4076387190.py:3: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_test = pd.read_csv(r"Document\test_data.csv")


In [4]:

column_chose=["Area","Q1_OccStatus","Q1_Occ_Year","Q1_OCC_A","Q1_OCC_A_Month","Q2","Q4_1","Q4_2","Q4_3","Q5_1","Q5_2","Q5_2A","Q5_2B",
              "Q5_3","Sex","Age","Status","Child",'Edu',"Salary","Live","Owner_loan",'Q5_5','Q5_6',]

#'SubDistrict','District','Province'
column_drop =['OtherAppSpec','OtherApp','Twitter','Youtube','Tiktok','Instragram',
               'Facebook','Line','Line_Level',
]  
# df_train=df_train[column_chose]
# df_test=df_test[column_chose]
df_train['label'] = df_train_target['Car'] 


df_train=df_train.drop(columns=column_drop)
df_test=df_test.drop(columns=column_drop)




In [5]:
df_train=df_train.iloc[:,1:].replace({888: np.NaN, 999: np.NaN,'888': np.NaN, '999': np.NaN})
df_test =df_test.iloc[:,1:].replace({888: np.NaN, 999: np.NaN,'888': np.NaN, '999': np.NaN})



In [6]:
r_columns_Na= [ x for x in df_train if df_train[x].isnull().sum() > 10000]


for x in r_columns_Na :
    print(x, df_train[x].isnull().sum())


Q1_IndBD_2 20825
Q1_IndBD_3 20952
Q1_OCC_A_Month 20474
Q1_Occ_Have 15943
Q2_OccNum 19628
Q2.1_OccStatus 19622
Q2.1_Occ 19623
Q2.1_OccCode 19622
Q2.1_OccInd 19622
Q2.1A 19738
Q2.2_OccStatus 20940
Q2.2_Occ 20940
Q2.2_OccCode 20940
Q2.2_OccInd 20940
Q2.2A 20941
Q3.1Year 14298
Q3.1LessYear 14290
Q3.1Month 14290
Q3.2Year 14484
Q3.2LessYear 14291
Q3.2Month 14348
Q3.3Reason 14381
Q4_7Other 20388
Q5_9Other 20911
Q7_5Month 20849
Q8_3A 19146
Q8_3B 17402
Q9_9Other 20939
Q10.1_3 17461
Q10.1_4 20304
Q10.1_5 20844
Q10.2_2 16047
Q10.2_3 20115
Q10.2_4 20818
Q10.2_5 20937
Q10.3_Num 14621
Q10.3_Percent 14670
LiveOther 20915
Owner_loan 10492
Owner_rent 10521
AssetOther 20927


In [7]:

df_train=df_train.drop(columns=r_columns_Na)
df_test=df_test.drop(columns=r_columns_Na)


In [8]:
df_train.to_csv('df_train.csv',index=False)
df_test.to_csv('df_test.csv',index=False)
train_data = TabularDataset('df_train.csv')
test = TabularDataset('df_test.csv')
train_data

,Area,SubDistrict,District,Province,Region,Q1_OccStatus,Q1_Occ,Q1_OccCode,Q1_OccInd,Q1_IndBD_1,...,Age,Status,Child,Edu,Salary,Live,Owner,Moblie,NoSocial,label
0,1,หนองบอน,ประเวศ,กรุงเทพฯ,1,3,รับจ้างทำความสะอาด,L15,L,NaN,...,32,1.0,NaN,3.0,3.0,5.0,2.0,1.0,0.0,0
1,2,เทศบาลตำบลนครชัยศรี,นครชัยศรี,นครปฐม,2,1,ขายทอดมัน,D4,D,BD1,...,45,2.0,2.0,1.0,6.0,3.0,1.0,0.0,1.0,0
2,1,บางซื่อ,บางซื่อ,กรุงเทพฯ,1,3,รับจ้างขายสลากกินแบ่งรัฐบาล,D4,D,BD26,...,45,2.0,1.0,3.0,2.0,3.0,1.0,1.0,0.0,0
3,2,หาดใหญ่,เทศบาลนครหาดใหญ่,สงขลา,5,3,รับจ้างงานก่อสร้าง,C3,C,NaN,...,40,2.0,1.0,4.0,3.0,3.0,2.0,0.0,1.0,0
4,2,ในเมือง,เทศบาลเมืองบ้านไผ่,ขอนแก่น,4,3,รับทำความสะอาดบ้านอาคาร,L15,L,NaN,...,52,2.0,3.0,1.0,2.0,3.0,1.0,0.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20961,2,หอรัตนไชย,เทศบาลนครพระนครศรีอยุธยา,พระนครศรีอยุธยา,2,1,ขายผักสด,D4,D,BD1,...,29,1.0,NaN,5.0,3.0,2.0,2.0,NaN,NaN,0
20962,2,ในเมือง,เทศบาลเมืองหนองคาย,หนองคาย,4,1,ขายไก่ทอด,D4,D,BD1,...,44,2.0,1.0,4.0,3.0,3.0,1.0,0.0,1.0,0
20963,2,พนัสนิคม,เทศบาลเมืองพนัสนิคม,ชลบุรี,2,1,ขายเสื้อผ้า,D3,D,BD5,...,42,2.0,1.0,5.0,4.0,3.0,2.0,NaN,NaN,1
20964,2,ในเมือง,เทศบาลเมืองบุรีรัมย์_1,บุรีรัมย์,4,1,นวดเพื่อสุขภาพ,P3,P,NaN,...,53,2.0,2.0,3.0,2.0,3.0,2.0,1.0,0.0,0


In [66]:



nn_options = {  # specifies non-default hyperparameter values for neural network models
    'num_epochs': 10,  # number of training epochs (controls training time of NN models)
    'learning_rate': ag.space.Real(1e-4, 1e-2, default=5e-4, log=True),  # learning rate used in training (real-valued hyperparameter searched on log-scale)
    'activation': ag.space.Categorical('relu', 'softrelu', 'tanh'),  # activation function used in NN (categorical hyperparameter, default = first entry)
    'dropout_prob': ag.space.Real(0.0, 0.5, default=0.1),  # dropout probability (real-valued hyperparameter)
}

gbm_options = {  # specifies non-default hyperparameter values for lightGBM gradient boosted trees
    'num_boost_round': 100,  # number of boosting rounds (controls training time of GBM models)
    'num_leaves': ag.space.Int(lower=26, upper=66, default=36),  # number of leaves in trees (integer hyperparameter)
}

hyperparameters = {  # hyperparameters of each model type
                   'GBM': gbm_options,
                   'NN_TORCH': nn_options,  # NOTE: comment this line out if you get errors on Mac OSX
                  } 
time_limit = 5*60  # train various models for ~2 min
num_trials = 1  # try at most 5 different hyperparameter configurations for each type of model
search_strategy = 'auto'  # to tune hyperparameters using random search routine with a local scheduler

hyperparameter_tune_kwargs = {  # HPO is not performed unless hyperparameter_tune_kwargs is specified
    'num_trials': num_trials,
    'scheduler' : 'local',
    'searcher': search_strategy,
}

label = 'label'
save_path = 'agModels-predictClass'  # specifies folder to store trained models
# At most 0.05 ms per row (20000 rows per second throughput)
infer_limit = 0.00005
# adhere to infer_limit with batches of size 10000 (batch-inference, easier to satisfy infer_limit)
infer_limit_batch_size = 10000

#num_bag_folds=15, num_bag_sets=6, num_stack_levels=8,


predictor = TabularPredictor(label=label, path=save_path).fit(train_data, presets='best_quality', time_limit=time_limit,
num_bag_folds=10, num_bag_sets=5, num_stack_levels=2,
infer_limit=infer_limit,
infer_limit_batch_size=infer_limit_batch_size,
excluded_model_types=['KNN','NN_TORCH'] ,
#refit_full=True,
 
#hyperparameters=hyperparameters ,
hyperparameter_tune_kwargs=hyperparameter_tune_kwargs
)

No model was trained during hyperparameter tuning NeuralNetFastAI_BAG_L3... Skipping this model.
Hyperparameter tuning model: XGBoost_BAG_L3 ... Tuning model for up to 8.57s of the 23.76s of remaining time.
  0%|          | 0/1 [00:00<?, ?it/s]	Fitting 10 child models (S1F1 - S1F10) | Fitting with ParallelLocalFoldFittingStrategy
	Stopping HPO to satisfy time limit...
  0%|          | 0/1 [00:13<?, ?it/s]
Fitted model: XGBoost_BAG_L3\T1 ...
	0.794	 = Validation score   (accuracy)
	13.56s	 = Training   runtime
	0.0s	 = Validation runtime
	0.148ms	 = Validation runtime (1 row | 10000 batch size | MARGINAL)
	0.462ms	 = Validation runtime (1 row | 10000 batch size)
Fitting model: LightGBMLarge_BAG_L3 ... Training model for up to 8.57s of the 10.0s of remaining time.
	Fitting 10 child models (S1F1 - S1F10) | Fitting with ParallelLocalFoldFittingStrategy
	0.791	 = Validation score   (accuracy)
	6.99s	 = Training   runtime
	0.22s	 = Validation runtime
	0.066ms	 = Validation runtime (1 row | 1

In [67]:
# predictor = TabularPredictor.load(save_path)  # unnecessary, just demonstrates how to load previously-trained predictor from file
# refit_model_map = predictor.refit_full()

# predictor.refit_full()
# predictor.persist_models()
# predictor.leaderboard(silent=True)


y_pred = predictor.predict(test)

df_submit = pd.read_csv('Document\sample_submission.csv')
df_submit['Predicted'] = y_pred


df_submit.to_csv('AutoGluon_5min_repleceNa_more_new_1.csv',index=False)
results = predictor.fit_summary(show_plot=True)

*** Summary of fit() ***
Estimated performance of each model:
                      model  score_val  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0         XGBoost_BAG_L2\T1   0.796194       0.744017  48.979827                0.000000          16.922395            2       True         18
1       WeightedEnsemble_L3   0.796194       0.773017  49.111824                0.029000           0.131996            3       True         20
2         XGBoost_BAG_L3\T1   0.794000       0.744017  59.632283                0.000000          13.559815            3       True         28
3      LightGBMXT_BAG_L1\T1   0.793904       0.000000  10.753694                0.000000          10.753694            1       True          1
4       WeightedEnsemble_L2   0.793904       0.027000  12.139691                0.027000           1.385996            2       True         10
5        CatBoost_BAG_L3\T1   0.793857       0.744017  58.892225                

In [235]:
!kaggle competitions submit -c hackathon-online-data-science-and-big-data -f AutoGluon_5min_repleceNa_more_new_1.csv -m "Message"